In [2]:
import pandas as pd
from pandas import datetime
import numpy as np
import math
import re
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:

data = pd.read_csv("C:/Users/shrini/Downloads/TCS.BO.csv")

In [4]:
tcs = data[["Date","Close"]]


In [6]:
tcs.head()


,Date,Close
0,2018-04-03,1454.050049
1,2018-04-04,1453.530029
2,2018-04-05,1478.569946
3,2018-04-06,1474.849976
4,2018-04-09,1460.569946


In [7]:
def add_datepart1(df, fldname, drop=True, time=False, errors="raise"):	

    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

def is_date(x): return np.issubdtype(x.dtype, np.datetime64)

In [8]:
tcs["Date"] = pd.to_datetime(tcs["Date"])
add_datepart1(tcs,'Date')
tcs.head()

C:\Users\shrini\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\shrini\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Close,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,1454.050049,2018,4,14,3,1,93,False,False,False,False,False,False,1522713600
1,1453.530029,2018,4,14,4,2,94,False,False,False,False,False,False,1522800000
2,1478.569946,2018,4,14,5,3,95,False,False,False,False,False,False,1522886400
3,1474.849976,2018,4,14,6,4,96,False,False,False,False,False,False,1522972800
4,1460.569946,2018,4,15,9,0,99,False,False,False,False,False,False,1523232000


In [9]:
tcs = tcs.dropna()
tcs.head()

,Close,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,1454.050049,2018,4,14,3,1,93,False,False,False,False,False,False,1522713600
1,1453.530029,2018,4,14,4,2,94,False,False,False,False,False,False,1522800000
2,1478.569946,2018,4,14,5,3,95,False,False,False,False,False,False,1522886400
3,1474.849976,2018,4,14,6,4,96,False,False,False,False,False,False,1522972800
4,1460.569946,2018,4,15,9,0,99,False,False,False,False,False,False,1523232000


,Date,Close
0,2018-04-03,8.840
1,2018-04-04,8.915
2,2018-04-05,8.935
3,2018-04-06,8.745
4,2018-04-09,8.630


In [11]:
#Train validation split
def train_test_split(a, n): return a[:n], a[n:]

n_val = 50
n_train = len(tcs)-n_val
X_train, X_val = train_test_split(tcs.drop(columns=["Close"]), n_train)
y_train, y_val = train_test_split(tcs["Close"], n_train)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((198, 13), (198,), (50, 13), (50,))

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap' : [False],'n_estimators': [3,10], 'max_features': [2, 3, 4]}
]

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [13]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 10}

In [14]:
cv_scores = grid_search.cv_results_
for mean_score, params in zip(cv_scores["mean_test_score"], cv_scores["params"]):
  print(np.sqrt(-mean_score), params)

130.08164767029285 {'max_features': 2, 'n_estimators': 3}
115.56133283001535 {'max_features': 2, 'n_estimators': 10}
101.43737350684864 {'max_features': 2, 'n_estimators': 30}
147.52066387750455 {'max_features': 4, 'n_estimators': 3}
128.1193547012908 {'max_features': 4, 'n_estimators': 10}
104.42013731782342 {'max_features': 4, 'n_estimators': 30}
112.38304160805819 {'max_features': 6, 'n_estimators': 3}
98.7877304023387 {'max_features': 6, 'n_estimators': 10}
102.72516893851699 {'max_features': 6, 'n_estimators': 30}
105.44729851918933 {'max_features': 8, 'n_estimators': 3}
100.73542740697292 {'max_features': 8, 'n_estimators': 10}
100.01965274711107 {'max_features': 8, 'n_estimators': 30}
111.08396953766491 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
108.7296653331833 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
141.40543034104562 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
111.43271758988466 {'bootstrap': False, 'max_features': 3, 'n_est

In [14]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression().fit(X_train,y_train)
reg.score(X_train,y_train)

0.4679461175865742

Lasso

In [15]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train,y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [17]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'alpha': [0.01,0.10,1.0], 'max_iter': [200,400,600,800]},
    {'normalize' : [True],'alpha': [0.01,0.10,1.0], 'max_iter': [200,400,600,800]}
]

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to in

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to in

GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [0.01, 0.1, 1.0], 'max_iter': [200, 400, 600, 800]}, {'normalize': [True], 'alpha': [0.01, 0.1, 1.0], 'max_iter': [200, 400, 600, 800]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [18]:
grid_search.best_params_

{'alpha': 1.0, 'max_iter': 800}

In [19]:
cv_scores = grid_search.cv_results_
for mean_score, params in zip(cv_scores["mean_test_score"], cv_scores["params"]):
  print(np.sqrt(-mean_score), params)

193.34202781928934 {'alpha': 0.01, 'max_iter': 200}
194.2757844772466 {'alpha': 0.01, 'max_iter': 400}
195.39978795944998 {'alpha': 0.01, 'max_iter': 600}
196.6941104237667 {'alpha': 0.01, 'max_iter': 800}
192.95624846737806 {'alpha': 0.1, 'max_iter': 200}
193.5105943481795 {'alpha': 0.1, 'max_iter': 400}
194.105094588869 {'alpha': 0.1, 'max_iter': 600}
194.71243519148877 {'alpha': 0.1, 'max_iter': 800}
188.67470844797688 {'alpha': 1.0, 'max_iter': 200}
186.4482857598098 {'alpha': 1.0, 'max_iter': 400}
184.23315686637318 {'alpha': 1.0, 'max_iter': 600}
182.503837708933 {'alpha': 1.0, 'max_iter': 800}
191.40894569578182 {'alpha': 0.01, 'max_iter': 200, 'normalize': True}
191.4150534482485 {'alpha': 0.01, 'max_iter': 400, 'normalize': True}
191.40857429027247 {'alpha': 0.01, 'max_iter': 600, 'normalize': True}
191.41307621407591 {'alpha': 0.01, 'max_iter': 800, 'normalize': True}
190.39568454406606 {'alpha': 0.1, 'max_iter': 200, 'normalize': True}
190.1149089356979 {'alpha': 0.1, 'max_i

Ridge

In [20]:
from sklearn.linear_model import Ridge


In [21]:
model_lasso = Ridge()

In [24]:
clf1 = linear_model.Ridge(alpha=0.1)
clf1.fit(X_train,y_train)

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.654860e-18
  overwrite_a=True).T


Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [25]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'alpha': [0.01,0.10,1.0], 'max_iter': [200,400,600,800]},
    {'normalize' : [True],'alpha': [0.01,0.10,1.0], 'max_iter': [200,400,600,800]}
]

grid_search = GridSearchCV(clf1, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.663722e-18
  overwrite_a=True).T
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.931568e-19
  overwrite_a=True).T
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.729857e-19
  overwrite_a=True).T
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.980524e-19
  overwrite_a=True).T
C:\U

C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.629325e-17
  overwrite_a=True).T
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.417656e-17
  overwrite_a=True).T
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.709352e-17
  overwrite_a=True).T
C:\Users\shrini\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.629325e-17
  overwrite_a=True).T
C:\U

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [0.01, 0.1, 1.0], 'max_iter': [200, 400, 600, 800]}, {'normalize': [True], 'alpha': [0.01, 0.1, 1.0], 'max_iter': [200, 400, 600, 800]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [26]:
grid_search.best_params_


{'alpha': 1.0, 'max_iter': 600, 'normalize': True}

In [27]:
cv_scores = grid_search.cv_results_
for mean_score, params in zip(cv_scores["mean_test_score"], cv_scores["params"]):
  print(np.sqrt(-mean_score), params)

4740.707394564728 {'alpha': 0.01, 'max_iter': 200}
4740.70740018896 {'alpha': 0.01, 'max_iter': 400}
4740.70739209991 {'alpha': 0.01, 'max_iter': 600}
4740.707394564728 {'alpha': 0.01, 'max_iter': 800}
925.389213131196 {'alpha': 0.1, 'max_iter': 200}
925.389212777705 {'alpha': 0.1, 'max_iter': 400}
925.389213131196 {'alpha': 0.1, 'max_iter': 600}
925.389213131196 {'alpha': 0.1, 'max_iter': 800}
167.84417693368852 {'alpha': 1.0, 'max_iter': 200}
167.84417694173652 {'alpha': 1.0, 'max_iter': 400}
167.84417693780236 {'alpha': 1.0, 'max_iter': 600}
167.84417693780236 {'alpha': 1.0, 'max_iter': 800}
173.65126529289702 {'alpha': 0.01, 'max_iter': 200, 'normalize': True}
173.65126529289398 {'alpha': 0.01, 'max_iter': 400, 'normalize': True}
173.65126529289512 {'alpha': 0.01, 'max_iter': 600, 'normalize': True}
173.65126529289702 {'alpha': 0.01, 'max_iter': 800, 'normalize': True}
171.3204217075831 {'alpha': 0.1, 'max_iter': 200, 'normalize': True}
171.32042170758749 {'alpha': 0.1, 'max_iter':